In [46]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install --upgrade selenium
!pip install gspread oauth2client
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# Install required libraries for Google Sheets and authentication
!pip install --upgrade gspread oauth2client google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from selenium.webdriver.remote import webelement

from IPython.display import Image, display
import os
import sys
from time import sleep

import pandas as pd
import gspread
from google.auth import default
from google.auth.exceptions import RefreshError

In [2]:

# Create a WebDriver instance
driver = webdriver.Chrome()

# Open a website to verify
driver.get('https://www.google.com')
# lấy tiêu đề của trang web
print(driver.title)



The chromedriver version (129.0.6668.89) detected in PATH at D:\PYTHON\Scripts\chromedriver.exe might not be compatible with the detected chrome version (130.0.6723.59); currently, chromedriver 130.0.6723.116 is recommended for chrome 130.*, so it is advised to delete the driver in PATH and retry


Google


In [49]:
# lấy thông tin theo xpath
print(driver.find_element(By.XPATH, '/html/body/div[1]/div[6]/div[1]').text)

Việt Nam


In [3]:
def handle_code_verification(driver: webdriver.Chrome) -> None:
    """
    Xử lý việc xác minh tài khoản bằng mã (nếu có).

    Args:
        driver (webdriver.Chrome): Trình duyệt Chrome WebDriver.
    """
    try:
        verification_prompt = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, 'xpath_to_verification_element'))
        )
        print("Hãy nhập mã xác minh thủ công trên trình duyệt.")
        WebDriverWait(driver, 120).until_not(
            EC.presence_of_element_located((By.XPATH, 'xpath_to_verification_element'))
        )
    except TimeoutException:
        print("Không có xác minh tài khoản hoặc đã hết thời gian chờ.")

def handle_cookie_acceptance(driver: webdriver.Chrome) -> None:
    """
    Xử lý việc chấp nhận cookies (nếu xuất hiện).

    Args:
        driver (webdriver.Chrome): Trình duyệt Chrome WebDriver.
    """
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, 'xpath_to_cookie_button'))
        )
        cookie_button.click()
    except TimeoutException:
        print("Không thấy hộp thoại cookies xuất hiện.")


In [4]:
def login_linkedin(driver: webdriver.Chrome, username: str, password: str) -> None:
    """
    Đăng nhập LinkedIn với username và password.

    Args:
        driver (webdriver.Chrome): Trình duyệt Chrome WebDriver.
        username (str): Tài khoản LinkedIn.
        password (str): Mật khẩu LinkedIn.
    """
    login_url = "https://www.linkedin.com/login"
    driver.get(login_url)

    try:
        # Điền username
        username_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="username"]'))
        )
        username_field.send_keys(username)

        # Điền password
        password_field = driver.find_element(By.XPATH, '//*[@id="password"]')
        password_field.send_keys(password)

        # Nhấn nút đăng nhập
        login_button = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[4]/button')
        login_button.click()
    except TimeoutException:
        print("Không tìm thấy các phần tử đăng nhập trong thời gian quy định.")
        return

    # Xử lý các bước tiếp theo nếu có
    try:
        handle_code_verification(driver)
        handle_cookie_acceptance(driver)
    except Exception as e:
        print(f"Lỗi trong quá trình xử lý thêm: {str(e)}")

    time.sleep(5)  # Đợi đăng nhập hoàn tất (nếu cần thiết)



In [5]:
username ="chidinhp4@gmail.com"
password = "Dinh0703"
login_linkedin(driver,username,password)

Không có xác minh tài khoản hoặc đã hết thời gian chờ.
Không thấy hộp thoại cookies xuất hiện.


In [36]:
import gspread
from google.oauth2.service_account import Credentials

# Đường dẫn đến tệp JSON của bạn
SERVICE_ACCOUNT_FILE = 'snappy-charge-444008-u3-c675ab8bd34f.json'

# Phạm vi quyền truy cập
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Xác thực với Google Sheets API
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
gc = gspread.authorize(credentials)

# Mở Google Sheet bằng URL
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1bhW3Cxj-yy-516yXMcs1SKxxgRj9WVOuZUFTxu8JKQ4/edit#gid=0'
sheet = gc.open_by_url(spreadsheet_url)

# Chọn worksheet cụ thể
work_sheet = sheet.worksheet('Sheet1')

# Lấy hàng tiêu đề
header = work_sheet.row_values(1)

# Kiểm tra sự tồn tại của các tiêu đề cần thiết
required_columns = ["Company name", "Title job", "Location job", "Salary"]
for column in required_columns:
    if column not in header:
        print(f"Column '{column}' not found in the header.")
        exit(1)

# Tìm vị trí các cột theo tiêu đề
company_col = header.index("Company name") + 1  # Cột "Company name"
job_col = header.index("Title job") + 1         # Cột "Title job"
locate_col = header.index("Location job") + 1  # Cột "Location job"
salary_col = header.index("Salary") + 1        # Cột "Salary"

# Kiểm tra kết nối bằng cách in tiêu đề
print(f"Successfully accessed sheet: {sheet.title}")
print("Headers:", header)

# Đọc tất cả dữ liệu
data = work_sheet.get_all_records()
print("Dữ liệu từ Google Sheet:", data)


Successfully accessed sheet: Crawling info job LinkedIn
Headers: ['Link job', 'Company name', 'Title job', 'Salary', 'Location job']
Dữ liệu từ Google Sheet: [{'Link job': 'https://www.linkedin.com/jobs/view/4086227901/', 'Company name': 'Bitcoin Builders', 'Title job': 'Senior AI Engineer (ML/Smart Contract/Python)', 'Salary': 'None', 'Location job': 'Ho Chi Minh City, Vietnam'}, {'Link job': 'https://www.linkedin.com/jobs/view/phd-applied-science-intern-data-science-at-linkedin-4081050265', 'Company name': 'LinkedIn', 'Title job': 'PhD Applied Science Intern- Data Science ', 'Salary': 'None', 'Location job': 'Mountain View, CA'}, {'Link job': 'https://www.linkedin.com/jobs/view/postdoctoral-researcher-fair-phd-at-meta-4078804658', 'Company name': 'Meta', 'Title job': 'Postdoctoral Researcher - FAIR (PhD) ', 'Salary': 'None', 'Location job': 'Menlo Park, CA'}, {'Link job': 'https://www.linkedin.com/jobs/view/phd-software-engineer-phd-early-career-systems-and-infrastructure-2025-start-

In [29]:
def extract_profile_data(driver: webdriver.Chrome, url: str):
    """
    Trích xuất dữ liệu hồ sơ từ trang LinkedIn.

    Args:
        driver (webdriver.Chrome): Đối tượng Selenium WebDriver.
        url (str): URL của trang LinkedIn cần quét.

    Returns:
        List: Dữ liệu hồ sơ gồm Company, Job Title, Location, Salary.
    """
    profiles_data = []
    try:
        driver.get(url)
        
        # Chờ nội dung trang tải xong
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        
        # Trích xuất dữ liệu hồ sơ
        try:
            company = driver.find_element(By.CSS_SELECTOR, '.job-details-jobs-unified-top-card__company-name').text
        except NoSuchElementException:
            company = "Unknown"

        try:
            job = driver.find_element(By.CSS_SELECTOR, '.job-details-jobs-unified-top-card__job-title').text
        except NoSuchElementException:
            job = "Unknown"

        try:
            locate = driver.find_element(By.CSS_SELECTOR, '.job-details-jobs-unified-top-card__primary-description-container span.tvm__text.tvm__text--low-emphasis').text
        except NoSuchElementException:
            locate = "Unknown"

        try:
            sal = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[2]/div/div/main/div[2]/div[1]/div/div[1]/div/div/div/div[4]/ul/li[1]/span/span[1]").text
            if '$' not in sal:
                sal = 'None'
        except NoSuchElementException:
            sal = "None"
        
        # In dữ liệu đã trích xuất
        print("Company:", company)  
        print("Job Title:", job)
        print("Location:", locate)
        print("Salary:", sal)
        
        # Thêm dữ liệu vào danh sách
        profiles_data.append([company, job, locate, sal])
        print(f"Trích xuất dữ liệu từ URL {url} thành công.")

    except TimeoutException:
        print(f"Trang không tải được: {url}")

    except Exception as e:
        print(f"Lỗi không xác định khi xử lý URL {url}: {e}")

    return profiles_data


In [30]:
# tìm theo css selector
print(driver.find_element(By.CSS_SELECTOR, '.job-details-jobs-unified-top-card__company-name').text)
print(driver.find_element(By.CSS_SELECTOR, '.job-details-jobs-unified-top-card__job-title').text)
city_info = driver.find_element(By.CSS_SELECTOR, '.job-details-jobs-unified-top-card__primary-description-container span.tvm__text.tvm__text--low-emphasis').text
print(city_info)  # In ra "Ho Chi Minh City, Vietnam"

Meta
Postdoctoral Researcher - FAIR (PhD) 
Menlo Park, CA


In [37]:
def update_google_sheet(work_sheet, data, start_row):
    """
    Cập nhật Google Sheet với dữ liệu hồ sơ LinkedIn đã trích xuất.

    Args:
        work_sheet: Đối tượng worksheet từ Google Sheets API.
        data: Danh sách dữ liệu để cập nhật (mỗi hàng là một danh sách con).
        start_row: Hàng bắt đầu để cập nhật dữ liệu (số nguyên).
    """
    # Đảm bảo dữ liệu là danh sách chứa danh sách con
    if isinstance(data[0], str):  
        formatted_data = [data]  # Bọc data vào danh sách nếu là 1 dòng đơn lẻ
    else:
        formatted_data = data

    # Xác định phạm vi cần cập nhật
    end_row = start_row + len(formatted_data) - 1
    range_to_update = f"B{start_row}:E{end_row}"  # Cột B đến E (4 cột dữ liệu)

    try:
        # Cập nhật dữ liệu vào Google Sheet
        work_sheet.update(range_name=range_to_update, values=formatted_data)
        print(f"Successfully updated row {start_row} with data: {formatted_data}")
    except Exception as e:
        print(f"Error updating Google Sheet: {e}")


In [33]:
link_col_index = header.index("Link job") + 1  # Index bắt đầu từ 0 nên cần +1
company_col_index = header.index("Company name") + 1  # Index bắt đầu từ 0 nên cần +1

# Lấy toàn bộ dữ liệu từ cột "Link" và "Company"
link_column_data = work_sheet.col_values(link_col_index)
company_column_data = work_sheet.col_values(company_col_index)
print(len(link_column_data))
# Duyệt qua và chỉ lấy các giá trị LinkedIn từ cột "Link" khi cột "Company" trống
linkedin_links = [
    (i + 1, link_column_data[i])
    for i in range(1, len(link_column_data))  # Bắt đầu từ 1 để bỏ qua header
]
print(linkedin_links)
# In ra số thứ tự và link hợp lệ
for row_number, link in linkedin_links:
    print(f"Row {row_number}: {link}")

255
[(2, 'https://www.linkedin.com/jobs/view/4086227901/'), (3, 'https://www.linkedin.com/jobs/view/phd-applied-science-intern-data-science-at-linkedin-4081050265'), (4, 'https://www.linkedin.com/jobs/view/postdoctoral-researcher-fair-phd-at-meta-4078804658'), (5, 'https://www.linkedin.com/jobs/view/phd-software-engineer-phd-early-career-systems-and-infrastructure-2025-start-msca-at-google-4079687860'), (6, 'https://www.linkedin.com/jobs/view/phd-software-engineer-early-career-ai-machine-learning-2025-start-msca-at-google-4081328297'), (7, 'https://www.linkedin.com/jobs/view/product-engineering-architect-circuit-simulation-at-cadence-4023227968'), (8, 'https://www.linkedin.com/jobs/view/student-researcher-doubao-seed-foundation-model-ai-platform-2025-start-phd-at-bytedance-4078781951'), (9, 'https://www.linkedin.com/jobs/view/wireless-sensing-intern-phd-at-meta-4095120080?refId=Y3zVaA0poxxjbinO4Env4Q%3D%3D&trackingId=lgQOJhs5fwzHDfupJfd6SA%3D%3D'), (10, 'https://www.linkedin.com/jobs/v

In [34]:
for i in linkedin_links:
    # Extract profile data từ link
    profile_data = extract_profile_data(driver, i[1])  # i[1] là link từ Google Sheet
    
    # Bọc profile_data vào danh sách để cập nhật lên Google Sheet
    if profile_data:  # Đảm bảo dữ liệu không rỗng
        update_google_sheet(work_sheet, [profile_data], i[0])  # i[0] là số dòng
        print(f"Row {i[0]} updated with data: {profile_data}")
    else:
        print(f"Row {i[0]}: No data extracted.")

# Đóng trình duyệt
driver.quit()
print("Crawling and Google Sheet update completed successfully!")


Company: Bitcoin Builders
Job Title: Senior AI Engineer (ML/Smart Contract/Python)
Location: Ho Chi Minh City, Vietnam
Salary: None
Trích xuất dữ liệu từ URL https://www.linkedin.com/jobs/view/4086227901/ thành công.
Error updating Google Sheet: APIError: [400]: Invalid values[1][0]: list_value {
  values {
    string_value: "Bitcoin Builders"
  }
  values {
    string_value: "Senior AI Engineer (ML/Smart Contract/Python)"
  }
  values {
    string_value: "Ho Chi Minh City, Vietnam"
  }
  values {
    string_value: "None"
  }
}

Row 2 updated with data: [['Bitcoin Builders', 'Senior AI Engineer (ML/Smart Contract/Python)', 'Ho Chi Minh City, Vietnam', 'None']]
Company: LinkedIn
Job Title: PhD Applied Science Intern- Data Science 
Location: Mountain View, CA
Salary: None
Trích xuất dữ liệu từ URL https://www.linkedin.com/jobs/view/phd-applied-science-intern-data-science-at-linkedin-4081050265 thành công.
Error updating Google Sheet: APIError: [400]: Invalid values[2][0]: list_value {
  